# step01_get_doc_from_sources
- In task 3, text from different department must be extracted.
- However, it is difficult to find the department in ../preprocessing/01_data4finetune/SNUH_visit_2011to2020.
- Therefore, the preprocessing started from Excel file (../sourcedata/???.xlsx) containing the department.

In [ ]:
import glob
import os
import random

randseed = random.randint(1, 1000)
print("randseed: ", randseed)
rng = random.Random(randseed)

data_category = ["SNUH_visit_2011to2020"]

## heldout numbers

In [ ]:
pools = []
train_pool = "../preprocessing/01_data4finetune/pts_SNUH_visit_2011to2020_heldout_train.txt"
test_pool  = "../preprocessing/01_data4finetune/pts_SNUH_visit_2011to2020_heldout_test.txt"

def target_ptnums(target_file):
    target_pool = []
    file = open(target_file, "r")
    lines = file.readlines()
    for l in range(len(lines)):
        line = lines[l].strip("\n")
        ptnum = line.split("/")[-1].replace(".txt", "")
        target_pool.append(ptnum)
    file.close()
    
    return target_pool

pools_train = target_ptnums(train_pool)
pools_test  = target_ptnums(test_pool)
pools = pools_train + pools_test
print("len(pools): ", len(pools))
print(pools[:10])

# read excels

In [ ]:
import pandas as pd
import os
import glob
import preprocessing

sources = ['../sources/visits_2011to2020/*.xlsx']
docperfile = 50

In [ ]:
for s in range(len(sources)):
    outputdir = sources[s].split("/")[-2]
    output_directory = "./data/"+str(outputdir)
    print("output_directory: ", output_directory)
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    
    filenames = glob.glob(sources[s])
    filenames.sort()
    print(filenames)
    
    document = []
    for f in range(len(filenames)):
        reading_file = filenames[f].split("/")[-1]
        print("reading_file: ", reading_file)

        print("reading...")
        ws = pd.read_excel(filenames[f])

        print("processing...")
        
        print(str(f) + " / " + str(len(filenames)) + "..." + filenames[f])
        for row in range(len(ws)):
            if (row+1)%1000==0:
                print(str(row) + " / " + str(len(ws)))
            
            # 각 row 마다 처리
            outtext = []
            
            pt_id = ws['fake_id'][row]        
            date = ws['수진(진료)일'][row]
            dptment = ws["수진(퇴원포함)진료과"][row]

            
            if len(document)>0:
                if (date != date_last or dptment!=dptment_last or pt_id!=pt_id_last):
                    # 출력 문서
                    document = ["[dptment_start]---------------"] + [dptment_last] + ["[dptment_end]---------------"] +\
                               ["[doc_start]---------------"] + [date_last] + document + ["[doc_end]---------------"] +\
                               ["\n"]

                    target_folder = output_directory+"/"+str(((pt_id_last//docperfile)+1)*docperfile)
                    if not os.path.exists(target_folder):
                        os.makedirs(target_folder)
                    
                    zeros = "".join(["0"]*(8-len(str(pt_id_last))))
                    pt_id_str = zeros + str(pt_id_last)
                    
                    if pt_id_str in pools:
                        file = open(target_folder+"/"+pt_id_str+".txt", "a")
                        file.write("\n".join(document))
                        file.close()

                    document = []

            
            doctype = ws['서식명'][row]
            doctype = preprocessing.text_preprocessing(doctype)        
            doctype = "".join(doctype)
            doctype = doctype.replace("\n", " ")
#             print("doctype: ", doctype)
            
            section = ws['서식항목명'][row]
            section = preprocessing.text_preprocessing(section)
            section = "".join(section)
            section = section.replace("\n", " ")
#             print("section: ", section)
            
            content = ws['서식내용'][row]
            content = preprocessing.text_preprocessing(content)
#             print("content: ", content)
            
            
            document.append(section)
            for c in range(len(content)):
                content_sub = content[c].split("\n")
                for s in range(len(content_sub)):
                    if content_sub[s]!="":
                        document.append(content_sub[s].strip())
#             print("document: ", document)
            
            date_last = date
            dptment_last = dptment
            pt_id_last= pt_id
            
#             # test code
#             if row%10==0:
#                 break
    
#         print("document: ", document)
        if len(document)!=0:
            # 출력 문서
            document = ["[dptment_start]---------------"] + [dptment_last] + ["[dptment_end]---------------"] +\
                       ["[doc_start]---------------"] + [date_last] + document + ["[doc_end]---------------"] +\
                       ["\n"]

            target_folder = output_directory+"/"+str(((pt_id_last//docperfile)+1)*docperfile)
            if not os.path.exists(target_folder):
                os.makedirs(target_folder)
            
            zeros = "".join(["0"]*(8-len(str(pt_id_last))))
            pt_id_str = zeros + str(pt_id_last)

            if pt_id_str in pools:
                file = open(target_folder+"/"+pt_id_str+".txt", "a")
                file.write("\n".join(document))
                file.close()

            document = []

            
#         break